In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import glob, os

In [4]:
chars = [x.split("/")[-1] for x in glob.glob("./data/*")]
chars



['ni',
 'ma',
 'da',
 'jian',
 'mei',
 'ta2',
 'shi',
 'hen',
 'jia',
 'ying',
 'guo',
 'hao',
 'jia2',
 'zhu',
 'ta',
 'zai',
 'zhong',
 'wo']

In [12]:
for char in chars:
    print(char)
    files = sorted(glob.glob("./data/%s/*.png"%char))

    import imageio
    arr = imageio.imread(files[10])

    arr[:,:,3]

    def Calc_Photo_Diff(X,Y):
        arr = imageio.imread(X)
        # Split the three channels
        R = arr[:,:,0]
        G = arr[:,:,1]
        B = arr[:,:,2]
        grey_scaled = R*0.299 + G*0.587 + B*0.114
        grey_scaled = 255-grey_scaled

        arr = imageio.imread(Y)
        # Split the three channels
        R = arr[:,:,0]
        G = arr[:,:,1]
        B = arr[:,:,2]
        grey_scaled2 = R*0.299 + G*0.587 + B*0.114
        grey_scaled2 = 255-grey_scaled2

        return grey_scaled2 - grey_scaled, grey_scaled, grey_scaled2


    diff, grey_scaled, grey_scaled2 = Calc_Photo_Diff(files[2],files[5])


    diff_signal = []

    for i in range(len(files)-1):
        diff, grey_scaled, grey_scaled2 = Calc_Photo_Diff(files[i],files[i+1])
        diff_signal += [np.max(np.abs(diff))]

    plt.figure(figsize=[20,10])

    plt.plot(diff_signal)

    #plt.show()

    stroke_timing = [float(x.split("/")[-1].split("-")[-1][:-5]) for x in files]

    def Segment_Signal(X):
        i = 0
        rising_loc = []
        falling_loc = []
        thres = 0
        rising_edge = True

        while i < len(X):
            if rising_edge:
                if X[i] > 0:
                    rising_loc += [i]
                    rising_edge = False
            else:
                if X[i] == 0:
                    falling_loc += [i]
                    rising_edge = True
            i += 1
        return rising_loc,falling_loc

    rising_loc,falling_loc = Segment_Signal(diff_signal)

    

    rising_loc[1:]

    falling_loc[:-1]

    i = 0
    np.sum(stroke_timing[falling_loc[:-1][i]:rising_loc[1:][i]+1])

    pauses = []
    for i in range(len(rising_loc[1:])):
        pauses += [np.sum(stroke_timing[falling_loc[:-1][i]:rising_loc[1:][i]+1])]

    pauses

    falling_loc

    final_segmentation = []

    for i in range(len(pauses)):
        if pauses[i] >= 0.2:
            final_segmentation.append(falling_loc[i])
    final_segmentation.append(falling_loc[-1])

    final_segmentation

    from shutil import copyfile

    target_loc = "chars/%s/"%char

    if not os.path.exists(target_loc):
        os.makedirs(target_loc)

    i = 1
    copyfile(files[0], target_loc+"0.png")
    for ele in final_segmentation:

        copyfile(files[ele], target_loc+str(i)+".png")
        i+=1


    # Convert Stroke Images to CSV

    tartget_dir = target_loc
    files = sorted(glob.glob(tartget_dir+"*.png"))
    files

    templates = []
    for i in range(len(files)-1):
        x,y,z = Calc_Photo_Diff(files[i],files[i+1])
        templates.append((np.array(x) > 0)*1) #Import Grayscale subtraction and convert to BINARY map

    for template in range(len(templates)):
        np.savetxt(tartget_dir+str(template)+'.csv', templates[template].astype(np.int), fmt='%d', delimiter=',')




ni
ma
da
jian
mei
ta2
shi
hen
jia
ying


IndexError: list index out of range

In [6]:
final_segmentation

[18, 29, 37, 57, 71, 96]

In [9]:
templates[0].shape

(3, 250, 250)